# 量子ビットのハミルトニアンエミュレーション（詳細版）
最終更新: 2024/03/22

## ▶ 本資料の説明
本資料は任意なハミルトニアンを持つ量子ビットをシミュレーションするためのコードを載せている。

段階的に

① QuTipを使ったシミュレーション

② SciPyを使ったシミュレーション

③ ネイティブ実装のシミュレーション

というように、物理的な性質のみを意識したコードから数値計算の中身に関連するコードへと移行していく。

[本資料の簡易版](ODEsolver_simple.ipynb)

[スライド資料](main.pdf)

[ブラウザ版](https://quel-oss.github.io/qubit_emulator/)

その他の資料:

[1量子ビットの操作アニメーション](MicrowaveHamiltonian.ipynb)

[2量子ビットの操作アニメーション](MicrowaveHamiltonian2Q.ipynb)

[マイクロ波のパルス計画アニメーション（未完成）](MicrowavePulseManager.ipynb)

## ▶ セルの構成
### セル1〜3: QuTipを使ったシミュレーション
**特徴:** 物理レベルのことのみを考慮し設計できる。

セル1: 必要なライブラリのインポート

セル2: Transmon量子ビットの実装

セル3: 各量子ゲートの実行

### セル4〜6: SciPyを使ったシミュレーション
**特徴:** QuTipの中にあった微分方程式ソルバーを取り出している。

セル4: 必要なライブラリのインポート

セル5:  Transmon量子ビットの実装

セル6: 各量子ゲートの実行


### セル7〜12: ネイティブ実装のシミュレーション( 商差分形式 )
**特徴:** SciPyの内部計算を取り出し、NumPyのみ使って実装しているため、Cへの移行やハードウェア化が容易になる。

セル7: 必要なライブラリのインポート

セル8: 必要な関数の定義

セル9: Transmon量子ビットの実装

セル10: RXゲートの実行

セル11: RYゲートの実行

セル12: RZゲートの実行

### セル13〜16: ネイティブ実装のシミュレーション( Nordsieck表現 )
**特徴:** SciPyの内部計算を取り出し、NumPyのみ使って実装しているため、Cへの移行やハードウェア化が容易になる。

セル13: 必要なライブラリのインポート

セル14: 必要な関数の定義

セル15: Transmon量子ビットの実装

セル16: 各量子ゲートの実行

In [ ]:
####### QuTip シミュレーション ########
# セル1: 必要なライブラリのインポート

import numpy as np
from qutip import *
import matplotlib.pyplot as plt

In [ ]:
# セル2: Transmon量子ビットの実装

#Qubit パラメータ
f_q = 1500

#演算子の定義
N = 2
a = destroy(N) # 消滅演算子

#初期状態
ini_state = basis(N,0)   #　|0>

def u_op(Omega, omega, t, gamma,psi,samples=300):
    state = psi
    t_list = np.linspace(0, t, samples+1)
    w = np.zeros(len(t_list), dtype=np.float32)
    for x in range(len(t_list)):
        w[x] = Omega*np.sin(2*np.pi*x*omega*(t/samples)+gamma)
    a = destroy(2)
    H_q = 2*np.pi * 1500 * a.dag() * a
    H_d = 2*np.pi * (a + a.dag())

    result = sesolve(H_tot(w, H_q, H_d), state, t_list)
    return result.states

def H_tot(w, H_q, H_d):
    H_tot = [H_q, [H_d, np.real(w)]] 
    return H_tot


#### Matplotlib Viewer
def plot_qstate(y, duration):
    '''Plot the graph for time development of 1 qubit.
    Arguments: a complex array of qubit state change in time and total duration in float.'''
    tlist = np.linspace( 0, duration, len(y) )
    y_0r = np.zeros(len(y))
    y_0i = np.zeros(len(y))
    y_1r = np.zeros(len(y))
    y_1i = np.zeros(len(y))
    totalsum = np.zeros(len(y))

    for i in range(len(y)):
        y_0r[i] = y[i][0].real
        y_0i[i] = y[i][0].imag
        y_1r[i] = y[i][1].real
        y_1i[i] = y[i][1].imag
        totalsum[i] = abs(y[i][0]**2) + abs(y[i][1]**2)
    
    plt.rcParams['figure.figsize'] = [10, 4]
    fig = plt.figure()
    ax = plt.subplot(111)

    ax.plot(tlist, y_0r, label=r"$\alpha$ real")
    ax.plot(tlist, y_0i, label=r"$\alpha$ imag")
    ax.plot(tlist, y_1r, label=r"$\beta$ real")
    ax.plot(tlist, y_1i, label=r"$\beta$ imag")
    ax.plot(tlist, totalsum, label="Total prob")

    # legend outside
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #legend inside
    #ax.legend()

    ax.set_xlabel('time [us]')
    ax.set_ylabel('prob amplitude')

    plt.show() 

In [ ]:
# セル3: 各量子ゲートの実行

#Xgate
y = u_op(50,1500,0.01,0,ini_state) # 0.01は50,1500に対して試行錯誤してえられたちょうどπパルスに相当する値

#Ygate
# y = u_op(50,1500,0.01,-np.pi/2,ini_state)

#Xgate
# y = u_op(50,1500,0.01,0,ini_state)

#Hgate
#y = u_op(50,7500,0.005,0,ini_state,1000)

#H+Z(Virtual)
#y = u_op(50,7500,0.005,0,ini_state,500)
#y+= u_op(50,7500,0.005,-np.pi, y[-1], 500)
#y+= u_op(50,7500,0.005,0, y[-1], 500)

pulselength = 0.01
plot_qstate(y, pulselength)
#print(y[0], y[-1])

In [ ]:
####### SciPyを使ったシミュレーション ########
# セル4: 必要なライブラリのインポート

import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
# セル5: Transmon量子ビットの実装

def func(t,psi,Omega,omega,gamma):
    w = Omega*np.sin(2*np.pi*omega*t+gamma)
    H_q = [0,0,0,2*np.pi * omega]
    H_d = [0,2*np.pi,2*np.pi,0]
    
    a = (H_q[0] + w*H_d[0])
    b = (H_q[1] + w*H_d[1])
    c = (H_q[2] + w*H_d[2])
    d = (H_q[3] + w*H_d[3])
    
    psi1 = [psi[0]*a + psi[1]*b, psi[0]*c + psi[1]*d] 
    
    psi1[0] = psi1[0]*-1j
    psi1[1] = psi1[1]*-1j
    
    return [psi1[0],psi1[1]]

def u_op(Omega, omega, t, gamma,psi,samples=300):
    state = psi
    t_list = np.linspace(0, t, samples+1)
    result = solve(Omega, omega, gamma, state, t_list)
    return result

def solve(Omega, omega, gamma, psi0, tlist):
    r = scipy.integrate.ode(func)
    r.set_f_params(Omega, omega,gamma)
    r.set_integrator('zvode', method="adams", order=12, atol=1e-08, rtol=1e-06, nsteps=1000, first_step=0, min_step=0,max_step=0)
    r.set_initial_value(psi0, tlist[0])
    psi = []
    psi.append(psi0)
    for n in range(len(tlist)-1):
        dt = tlist[1]
        psi.append(r.integrate(r.t+dt))        
    return psi

#### Matplotlib Viewer
def plot_qstate(y, duration):
    '''Plot the graph for time development of 1 qubit.
    Arguments: a complex array of qubit state change in time and total duration in float.'''
    tlist = np.linspace( 0, duration, len(y) )
    y_0r = np.zeros(len(y))
    y_0i = np.zeros(len(y))
    y_1r = np.zeros(len(y))
    y_1i = np.zeros(len(y))
    totalsum = np.zeros(len(y))

    for i in range(len(y)):
        y_0r[i] = y[i][0].real
        y_0i[i] = y[i][0].imag
        y_1r[i] = y[i][1].real
        y_1i[i] = y[i][1].imag
        totalsum[i] = abs(y[i][0]**2) + abs(y[i][1]**2)
    
    plt.rcParams['figure.figsize'] = [10, 4]
    fig = plt.figure()
    ax = plt.subplot(111)

    ax.plot(tlist, y_0r, label=r"$\alpha$ real")
    ax.plot(tlist, y_0i, label=r"$\alpha$ imag")
    ax.plot(tlist, y_1r, label=r"$\beta$ real")
    ax.plot(tlist, y_1i, label=r"$\beta$ imag")
    ax.plot(tlist, totalsum, label="Total prob")

    # legend outside
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #legend inside
    #ax.legend()

    ax.set_xlabel('time [us]')
    ax.set_ylabel('prob amplitude')

    plt.show() 

In [ ]:
# セル6: 各量子ゲートの実行
ini_state = [1+0j,0]

#Xgate
y = u_op(50,1500,0.01,0,ini_state) # 0.01は50,1500に対して試行錯誤してえられたちょうどπパルスに相当する値

#Ygate
# y = u_op(50,1500,0.01,-np.pi/2,ini_state)

#Xgate
# y = u_op(50,1500,0.01,0,ini_state)

#Hgate
#y = u_op(50,7500,0.005,0,ini_state,1000)

#H+Z(Virtual)
#y = u_op(50,7500,0.005,0,ini_state,500)
#y+= u_op(50,7500,0.005,-np.pi, y[-1], 500)
#y+= u_op(50,7500,0.005,0, y[-1], 500)

pulselength = 0.01
plot_qstate(y, pulselength)
#print(y[0], y[-1])

In [ ]:
####### ネイティブ実装のシミュレーション ########
# セル7: 必要なライブラリのインポート

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# セル8: 必要な関数の定義

def factorial(n):
    '''Get the factorial of a integer n.
    Example:    factorial(3) => 6'''
    if ( not(isinstance(n, int)) or (np.sign(n)== -1) ):
        raise Exception('Error: positive integer expected as argument')
    else:
        res = 1
        while (n>1):
            res *= n
            n-= 1
        return res
        
def comb_cal(j):
    '''Calculate the combination in order to solve gamma_cal(j).
    Argument: positive integer j. Returns an array of floats.
    Example:    comb_cal(4) => [1, 6, 11, 6]'''
    if ( not(isinstance(j, int)) or (np.sign(j)== -1) ):
        raise Exception('Error: positive integer expected as argument')
    else:
        if(j == 0):
            return 1
        else:
            k = j
            vec_dict = {}
            while (k != 0):
                vec_dict[k] = np.zeros(k)
                k -= 1
            vec_dict[1][0] = 1
            if (j>1):
                for l in range(2,j+1):
                    for k in range(1,l+1):
                        if(k==1):
                            vec_dict[l][k-1] = 1
                        elif(k==l):
                            vec_dict[l][k-1] = (l-1)*vec_dict[l-1][k-2]
                        else:
                            vec_dict[l][k-1] = vec_dict[l-1][k-1] + (l-1)*vec_dict[l-1][k-2] 
            return vec_dict[j]

def comb_cal2(j):
    '''Calculate the combination in order to solve gamma_star_cal(j).
    Argument: positive integer j. Returns an array of floats.
    Example:    comb_cal2(4) =>  [1, 2, -1, -2, 0]'''
    if ( not(isinstance(j, int)) or (np.sign(j)== -1) ):
        raise Exception('Error: positive integer expected as argument')
    else:
        if(j == 0):
            return 1
        else:
            k = j+1
            vec_dict = {}
            while (k != 0):
                k -= 1
                vec_dict[k] = np.zeros(k+1)
            vec_dict[1][0] = 1
            if (j>0):
                for l in range(0,j+1): #1,2, (3(2+1)
                    for k in range(0,l+1): #0,1...
                        if(k==0):
                            vec_dict[l][k] = 1
                        elif(k==l):
                            if(k==1):
                                vec_dict[l][k] = (l-2)*vec_dict[l][k-1]
                            else:
                                vec_dict[l][k] = 0
                        else:
                            if(l<=1):
                                vec_dict[l][k] = vec_dict[l][k] + (l-2)*vec_dict[l][k-1] 
                            else:    
                                vec_dict[l][k] = vec_dict[l-1][k] + (l-2)*vec_dict[l-1][k-1]       
            return vec_dict[j]

def gamma_cal(j):
    '''Calculate the gamma value for an order j.
    Argument is a positive integer. Returns float.
    Example:    gamma_cal(1) => 0.5
    Limitation: j must be equal or smaller than 170'''
    if ( not(isinstance(j, int)) or (np.sign(j)== -1) ):
        raise Exception('Error: positive integer expected as argument')
    else:
        k = j
        arr = comb_cal(j)
        if (j==0):
            return arr
        gamma = 0
        while(j!=0):
            gamma += arr[k-j]/(j+1)
            j -= 1
        gamma /= factorial(k)
        return gamma

def gamma_star_cal(j):
    '''Calculate the gamma_star value for an order j.
    Argument is a positive integer. Returns float.
    Example:    gamma_star_cal(1) => -0.5
    Limitation: j must be equal or smaller than 170'''
    if ( not(isinstance(j, int)) or (np.sign(j)== -1) ):
        raise Exception('Error: positive integer expected as argument')
    else:
        k = j+1
        arr = comb_cal2(j)
        if (j==0):
            return arr
        gamma_star = 0
        while(k!=0):
            k -= 1
            gamma_star += arr[j-k]/(k+1)
        gamma_star /= factorial(j)
        return gamma_star

def nabla_func(f,X,Y,params,j,n):
    '''Calculate the nabla value of a function.
    Arguments: function f, function arguments X, Y, order j, number of valid steps n.
    Returns the numerical value in float.'''
    if (j==0):
        return ( f(X[n], Y[n], params ) ) 
    else:
        if ( hasattr( Y[0], '__len__' ) ):
            nabla = np.zeros(len(Y[0]), dtype="complex")
            for i in range( len(Y[0]) ):
                nabla[i] = nabla_func(f,X,Y,params,j-1,n)[i] - nabla_func(f,X,Y,params,j-1,n-1)[i]
            return nabla
        else:
            nabla = nabla_func(f,X,Y,params,j-1,n) - nabla_func(f,X,Y,params,j-1,n-1)
            return nabla 

def ddf_adams(f,X,Y,k,c,params=None):
    '''Calculate the Ordinal Differential Equation of a function with the Adams method, using Differential Division Format.
    Arguments: function f, array of function arguments X, Y, order j, correction steps cs.
    X has to be completely defined.
    Y must have at least one defined value, and the others values may be 0.
    Returns the updated array Y.'''
    
    h = X[1]-X[0] # stepsize
    s = len(X)-1 # number of total steps
    n = 0 # actual step
    k = k # number or previous steps in calculation
    c = c # number of corrections (ec)
    
    width = 1
    
    # prepare initial multistep
    while (n<k):
        res=nabla_func(f,X,Y,params,0,n)
        if ( hasattr(Y[0], '__len__') ):
            for i in range(len(Y[0])):
                Y[n+1][i] = Y[n][i] + h*res[i]
        else:
            Y[n+1] = Y[n] + h*res
        
        #implicit prepare test
        cs=0        
        while(cs<c):
            if ( hasattr(Y[0], '__len__') ):
                res = np.zeros(len(Y[0]), dtype="complex")
                nabla = nabla_func(f,X,Y,params,0,n+1)
                for i in range(len(Y[0])):
                    res[i] += gamma_star_cal(0) * nabla[i]
                    Y[n+1][i] = Y[n][i] + h*res[i]
                
            else:
                res = 0
                nabla = nabla_func(f,X,Y,params,0,n+1)
                res += gamma_star_cal(0) * nabla
                Y[n+1] = Y[n] + h*res
            cs += 1        
        n += 1       
        
    # multistep processing
    while (n<s):
        ##prediction
        if ( hasattr(Y[0], '__len__') ):
            res = np.zeros(len(Y[0]), dtype="complex")
            for j in range(k):
                nabla = nabla_func(f,X,Y,params,j,n)
                for i in range(len(Y[0])):
                    res[i] += gamma_cal(j) * nabla[i]    
            for i in range(len(Y[0])):
                Y[n+1][i] = Y[n][i] + h*res[i]
            
        else:
            res = 0
            for j in range(k):
                nabla = nabla_func(f,X,Y,params,j,n)
                res += gamma_cal(j) * nabla
            Y[n+1] = Y[n] + h*res
        
        ##estimation+correction
        cs=0 # actual correction step
        while(cs<c):
            if ( hasattr(Y[0], '__len__') ):
                res = np.zeros(len(Y[0]), dtype="complex")
                for j in range(k):
                    nabla = nabla_func(f,X,Y,params,j,n)
                    for i in range(len(Y[0])):
                        res[i] += gamma_cal(j) * nabla[i]    
                for i in range(len(Y[0])):
                    Y[n+1][i] = Y[n][i] + h*res[i]   
                    
            else:
                res = 0
                for j in range(k):
                    nabla = nabla_func(f,X,Y,params,j,n+1)
                res += gamma_star_cal(j) * nabla
                Y[n+1] = Y[n] + h*res
            cs += 1       
        n += 1
        
    return Y


def func(t,psi,params=None):  
    Omega = 50
    omega = 750
    omega_q = 900
    gamma = np.pi
    
    w = Omega*np.sin(2*np.pi*omega*t+gamma)
    H_q = [0,0,0,2*np.pi * omega_q]
    H_d = [0,2*np.pi,2*np.pi,0]
    
    a = (H_q[0] + w*H_d[0])
    b = (H_q[1] + w*H_d[1])
    c = (H_q[2] + w*H_d[2])
    d = (H_q[3] + w*H_d[3])
    
    psi1 = [psi[0]*a + psi[1]*b, psi[0]*c + psi[1]*d] 
    
    psi1[0] = psi1[0]*-1j
    psi1[1] = psi1[1]*-1j
    
    return [psi1[0],psi1[1]]

In [ ]:
# セル9: Transmon量子ビットの実装

def calc_qstate( qtype=["sc_transmon", 750], ini_state=[[1+0j,0j]], wave_params= [0.03, 1000, [50,750,0] ] ):
    '''Calculate the quantum state for a predefined hamiltonian to a qubits system, according to the initial state and the input wave parameters.
    Arguments: an array of qubit type in string and its frequency in float, a 2D array of the qubit system initial state, wave parameter in array [wave time, total steps, [amplitude, frequency, phase] ]
    Returns the qubit system state for each timestep '''
    
    # Prepare an array for the timesteps
    tlast, nsteps, params = wave_params
    tlist = np.linspace( 0, tlast, nsteps )    
   
    # Prepare an array for calculating y in all time steps
    y = [ [0j,0j] for i in range(len(tlist)) ]
    y[0] = ini_state[0]
    
    # Define the function of the specific qubit type
    if (qtype[0]=="sc_transmon"): # Transmon qubit. 2 elements state vector input-output.
        def func(t,psi,params):
            omega_q = qtype[1]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q = [0,0,0,2*np.pi * omega_q]
            H_d = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            psi1 = [ (psi[0]*U[0] + psi[1]*U[1]) *-1j, (psi[0]*U[2] + psi[1]*U[3]) *-1j ] 
            return psi1 
    if (qtype[0]=="sc_transmon_noisy"): # Noisy transmon. Density Matrix input-output. 作成中
        def func(t,rho,params):
            omega_q = qtype[1]
            anharm = qtype[2]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q = [0,0,0,2*np.pi * omega_q]
            H_d = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            rho1 = [ (rho[0]*U[0] + rho[1]*U[1]) *-1j, (rho[2]*U[0] + rho[1]*U[3]) *-1j, (rho[0]*U[2] + rho[1]*U[3]) *-1j, (rho[0]*U[2] + rho[1]*U[3]) *-1j ] 
            return rho1 
    if (qtype[0]=="sc_transmon_2q"): # 2 qubit transmon. 4 elements state vector input-output 作成中
        def func(t,psi,params):
            omega_q1 = qtype[1][0]
            omega_q2 = qtype[2][0]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q1 = [0,0,0,2*np.pi * omega_q]
            H_q2 = [0,0,0,2*np.pi * omega_q]
            H_d1 = [0,2*np.pi,2*np.pi,0]
            H_d2 = [0,2*np.pi,2*np.pi,0]
            H_int12 = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            psi1 = [ (psi[0]*U[0] + psi[1]*U[1]) *-1j, (psi[0]*U[2] + psi[1]*U[3]) *-1j ] 
            return psi1 
        
    # Calculate the Ordinal Differential Equation with Adams Method
    y=ddf_adams(func, tlist, y, k=5, c=1, params=params)
    
    return y
    
def plot_qstate(y, duration):
    '''Plot the graph for time development of 1 qubit.
    Arguments: a complex array of qubit state change in time and total duration in float.'''
    tlist = np.linspace( 0, duration, len(y) )
    y_0r = np.zeros(len(y))
    y_0i = np.zeros(len(y))
    y_1r = np.zeros(len(y))
    y_1i = np.zeros(len(y))
    totalsum = np.zeros(len(y))

    for i in range(len(y)):
        y_0r[i] = y[i][0].real
        y_0i[i] = y[i][0].imag
        y_1r[i] = y[i][1].real
        y_1i[i] = y[i][1].imag
        totalsum[i] = abs(y[i][0]**2) + abs(y[i][1]**2)
    
    plt.rcParams['figure.figsize'] = [10, 4]
    fig = plt.figure()
    ax = plt.subplot(111)

    ax.plot(tlist, y_0r, label=r"$\alpha$ real")
    ax.plot(tlist, y_0i, label=r"$\alpha$ imag")
    ax.plot(tlist, y_1r, label=r"$\beta$ real")
    ax.plot(tlist, y_1i, label=r"$\beta$ imag")
    ax.plot(tlist, totalsum, label="Total prob")

    # legend outside
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #legend inside
    #ax.legend()

    ax.set_xlabel('time [us]')
    ax.set_ylabel('prob amplitude')

    plt.show() 

In [ ]:
# セル10: RXゲートの実行

theta = np.pi

# --- #

# Rx( theta )
def rx( theta ):
    nsteps = int( 1000 * abs(theta)/(np.pi) ) #ステップ数. ある程度多くないと時間間隔が広すぎて発散してしまう
    amplitude = 50 # 設定値　信号の強度
    frequency = 1500 # 設定値　1500 [MHz]: 信号の周波数
    phase = 0 # 位相　
    pulselength = 0.01 * abs(theta)/(np.pi) # 0.01 [us]: amp=50, freq=1500の時にπ回転に必要な時間( QuTipで求めた実験値) 
    return ( pulselength, nsteps, [amplitude, frequency, phase] )

y = calc_qstate( qtype=["sc_transmon",1500], ini_state=[[1+0j,0j]], wave_params= rx( theta ) )
pulselength = rx( np.pi )[0]
plot_qstate( y, pulselength )

In [ ]:
# セル11: RYゲートの実行

theta = np.pi

# --- #

# Ry( theta )
def ry( theta ):
    nsteps = int( 1000 * abs(theta)/(np.pi) ) #ステップ数. ある程度多くないと時間間隔が広すぎて発散してしまう
    amplitude = 50 # 設定値　信号の強度
    frequency = 1500 # 設定値　1500 [MHz]: 信号の周波数
    phase = np.pi/2 # 位相　
    pulselength = 0.01 * abs(theta)/(np.pi) # 0.01 [us]: amp=50, freq=1500の時にπ回転に必要な時間( QuTipで求めた実験値) 
    return ( pulselength, nsteps, [amplitude, frequency, phase] )

y = calc_qstate( qtype=["sc_transmon",1500], ini_state=[[1+0j,0j]], wave_params= ry( theta ) )
pulselength = ry( np.pi )[0]
plot_qstate( y, pulselength )

In [ ]:
# セル12: RZゲートの実行

theta = 2*np.pi

# --- #

y = calc_qstate( qtype=["sc_transmon",1500], ini_state=[[.5**.5+0j,.5**.5+0j]], wave_params= [0.005 * theta/np.pi, int(1000* theta/np.pi), [50, 1500, -np.pi] ] )
y += calc_qstate( qtype=["sc_transmon",1500], ini_state=[y[-1]], wave_params=[0.005 * theta/np.pi, int(1000 * theta/np.pi), [50, 1500, 0] ]  )

pulselength = 0.01*theta/(np.pi)
plot_qstate( y, pulselength )

In [ ]:
####### ネイティブ実装のシミュレーション（Nordsieck表現） ########
# セル13: 必要なライブラリのインポート

import sympy as sp
import numpy as np
from scipy.special import bernoulli

In [ ]:
# セル14: 必要な関数の定義

def func(x, y):
    '''Function to be differentiated. Must return an expression in the form f(x,y). 
    Example1:    return [x+y**2] 
    Example2:    return [np.sin(2*y)**x] '''
    res = x+y**2
    return (res)


def derivative(func, order = 1):
    '''Get the derivative of a function in the first argument, and its order is determined in the second argument.
    Example:    derivative(x**2, 1) => 2*x '''
    if(order == 0):
        return func
    else:
        if(order>0):
            order -= 1
            y = derivative(func, order)
            x = sp.Symbol('x')
            res = y.diff(x)
            return (res)

def factorial(n):
    '''Get the factorial of a integer n.
    Example:    factorial(3) => 6'''
    res = 1
    while (n>1):
        res *= n
        n-= 1
    return res

def nordsieck_vec(func, order = 10, sym=False, x0=0, y0=0 ):
    '''Generate a Nordsieck vector, according to a given function and order.
    Results in dict format. Key: index. Value: corresponding value.
    1st argument: function to be solved
    2nd argument: order to be implemented
    3rd argument(optional): output in symbols, instead of numerical. '''
    z = {}
    for i in range(order):
        if (sym == True):
            z[i] = derivative(func, i)
        else:
            if (i==0):
                z[i] = y0
            else:
                z[i] = sp.lambdify(x, derivative(func, i), 'numpy')(0)
    return z


def combination(n,r):
    '''Get the combination of integers n and r.
    Example:    combination(2,3) => 0.3333333'''
    res = factorial(n)/(factorial(r)*factorial(n-r))
    return res

def pascal_triangle(n):
    '''Get the Pascal triangle for a given integer n in a square matrix size n x n.
    Output in Numpy Array.'''
    res = np.reshape(np.zeros(n**2), (n,n))
    for i in range(n):
        for j in range(n):
            if( (i<=j) ):
                res[i][j] = combination(j,i) 
    return res

def gen_M(l):
    '''Get the matrix M, used to update the Nordsieck vector with size n x n.
    Input is a n sized vector/array of constants l_0, l_1, ... l_n.'''
    n = len(l)
    e = np.zeros(n)
    e[1] = 1
    matrix = pascal_triangle(n)
    e_ = np.zeros(n)
    for i in range(n):
        for j in range(n):
            e_[j] += e[i]*matrix[i][j]
    lep = np.reshape(np.zeros(n**2), (n,n))
    for i in range(n):
        for j in range(n):
            lep[i][j] = l[i]*e_[j]
    res = matrix - lep
    return res

def next_nordsieck(z,M):
    '''Get the next Nordsieck Vector z_1 according to the current Nordsieck Vector z_0 and matrix M.'''
    n = len(z)
    res = np.zeros(n)
    for i in range(n):
        for j in range(n):
            res[i] += z[j]*M[i][j]
    return res

def gen_symM(n):
    '''Get the matrix M, used to update the Nordsieck vector with size n x n.
    Output is in symbolic format.'''
    x = []
    for i in range(n-1):
        x.append(sp.Symbol('x_'+str(i)))
                 
    l = []
    for i in range(n-1):
        l.append(x[i])
        if (i == 1-1):
            l.append(1)
                 
    e = sp.Matrix(np.zeros(n, dtype="int"))
    e[1] = 1
    
    matrix = sp.Matrix(pascal_triangle(n).astype(int))
    e_ = sp.Matrix(np.zeros(n, dtype="int"))

    for i in range(n):
        for j in range(n):
            e_[j] += e[i]*matrix[n*i+j]

    lep = sp.Matrix(np.reshape(np.zeros(n**2), (n,n)))
    for i in range(n):
        for j in range(n):
            lep[n*i+j] = sp.nsimplify(l[i]*e_[j])
                 
    res = matrix - lep
    return res

def get_l(n=4):
    '''Get the matrix M, used to update the Nordsieck vector with size n x n.
    (Only for n=2,3,4)
    Output is in symbolic format.'''
    l = np.ones(n)

    x = []
    for i in range(n-1):
        x.append(sp.Symbol('x_'+str(i)))

    A = gen_symM(n)

    # Get the Bernoulli numbers
    bnum = bernoulli(n-1)

    # PS: only for n<=4

    if (n==2):
        l[0] = 1/2 

    if (n==3):
        l[2] = 1/2

        # Solve remaining constants with condition Error Constant C = 0
        C = -sp.Matrix([[x[0],l[1],l[2]]])*sp.Matrix(bnum)/(factorial(0)*sp.Matrix( [x[0],l[1],l[2] ] )[0])
        l[0] = float(sp.Rational(sp.solve(C,x[0])[x[0]]))

    if (n==4):
        eq1 = sp.solve([ list(A.eigenvals().keys())[1],list(A.eigenvals().keys())[2] ], [x[2]], dict=True)[0][x[2]]-x[2]
        a = list(set(sp.sympify(list(A.eigenvals().keys())[1]).args)^set(sp.sympify(list(A.eigenvals().keys())[2]).args))
        eq2 = a[0]-a[1]

        # Solving the system of equations
        sol = sp.solve([eq1, eq2], [x[1], x[2]])

        l[2] = sol[0][0]
        l[3] = sol[0][1]

        # Solve remaining constants with condition Error Constant C = 0
        C = -sp.Matrix([[x[0],l[1],l[2],l[3]]])*sp.Matrix(bnum)/(factorial(0)*sp.Matrix( [x[0],l[1],l[2],l[3] ] )[0])
        l[0] = float(sp.Rational(sp.solve(C,x[0])[x[0]]))

    return l


In [ ]:
## セル15: Transmon量子ビットの実装 ######## 作成中

def calc_qstate( qtype=["sc_transmon", 750], ini_state=[[1+0j,0j]], wave_params= [0.03, 1000, [50,750,0] ] ):
    '''Calculate the quantum state for a predefined hamiltonian to a qubits system, according to the initial state and the input wave parameters.
    Arguments: an array of qubit type in string and its frequency in float, a 2D array of the qubit system initial state, wave parameter in array [wave time, total steps, [amplitude, frequency, phase] ]
    Returns the qubit system state for each timestep '''
    
    # Prepare an array for the timesteps
    tlast, nsteps, params = wave_params
    tlist = np.linspace( 0, tlast, nsteps )    
   
    # Prepare an array for calculating y in all time steps
    y = [ [0j,0j] for i in range(len(tlist)) ]
    y[0] = ini_state[0]
    
    # Define the function of the specific qubit type
    if (qtype[0]=="sc_transmon"): # Transmon qubit. 2 elements state vector input-output.
        def func(t,psi,params):
            omega_q = qtype[1]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q = [0,0,0,2*np.pi * omega_q]
            H_d = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            psi1 = [ (psi[0]*U[0] + psi[1]*U[1]) *-1j, (psi[0]*U[2] + psi[1]*U[3]) *-1j ] 
            return psi1 
    if (qtype[0]=="sc_transmon_noisy"): # Noisy transmon. Density Matrix input-output. 作成中
        def func(t,rho,params):
            omega_q = qtype[1]
            anharm = qtype[2]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q = [0,0,0,2*np.pi * omega_q]
            H_d = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            rho1 = [ (rho[0]*U[0] + rho[1]*U[1]) *-1j, (rho[2]*U[0] + rho[1]*U[3]) *-1j, (rho[0]*U[2] + rho[1]*U[3]) *-1j, (rho[0]*U[2] + rho[1]*U[3]) *-1j ] 
            return rho1 
    if (qtype[0]=="sc_transmon_2q"): # 2 qubit transmon. 4 elements state vector input-output 作成中
        def func(t,psi,params):
            omega_q1 = qtype[1][0]
            omega_q2 = qtype[2][0]
            Omega, omega_d, gamma = params
            w = Omega*np.sin(2*np.pi*omega_d*t+gamma)
            H_q1 = [0,0,0,2*np.pi * omega_q]
            H_q2 = [0,0,0,2*np.pi * omega_q]
            H_d1 = [0,2*np.pi,2*np.pi,0]
            H_d2 = [0,2*np.pi,2*np.pi,0]
            H_int12 = [0,2*np.pi,2*np.pi,0]
            U = (H_q[0] + w*H_d[0]),(H_q[1] + w*H_d[1]),(H_q[2] + w*H_d[2]),(H_q[3] + w*H_d[3])
            psi1 = [ (psi[0]*U[0] + psi[1]*U[1]) *-1j, (psi[0]*U[2] + psi[1]*U[3]) *-1j ] 
            return psi1 
        
    # Calculate the Ordinal Differential Equation with Adams Method
    x = sp.Symbol('x')
    y = sp.Symbol('y')

    #Nordsieck vector z0 in t0
    z0 = list(nordsieck_vec(func(tlist[0],y[0],params), 4, sym=False).values())
    l = get_l(4)
    M = gen_M(l)
    
    #Nordsieck vector z1 in t1
    z1 = next_nordsieck(z0,M)
    
    return z1
    
def plot_qstate(y, duration):
    '''Plot the graph for time development of 1 qubit.
    Arguments: a complex array of qubit state change in time and total duration in float.'''
    tlist = np.linspace( 0, duration, len(y) )
    y_0r = np.zeros(len(y))
    y_0i = np.zeros(len(y))
    y_1r = np.zeros(len(y))
    y_1i = np.zeros(len(y))
    totalsum = np.zeros(len(y))

    for i in range(len(y)):
        y_0r[i] = y[i][0].real
        y_0i[i] = y[i][0].imag
        y_1r[i] = y[i][1].real
        y_1i[i] = y[i][1].imag
        totalsum[i] = abs(y[i][0]**2) + abs(y[i][1]**2)
    
    plt.rcParams['figure.figsize'] = [10, 4]
    fig = plt.figure()
    ax = plt.subplot(111)

    ax.plot(tlist, y_0r, label=r"$\alpha$ real")
    ax.plot(tlist, y_0i, label=r"$\alpha$ imag")
    ax.plot(tlist, y_1r, label=r"$\beta$ real")
    ax.plot(tlist, y_1i, label=r"$\beta$ imag")
    ax.plot(tlist, totalsum, label="Total prob")

    # legend outside
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #legend inside
    #ax.legend()

    ax.set_xlabel('time [us]')
    ax.set_ylabel('prob amplitude')

    plt.show() 

In [ ]:
## セル16: 各量子ゲートの実行　######## 作成中

theta = np.pi

# --- #

# Rx( theta )
def rx( theta ):
    nsteps = int( 1000 * abs(theta)/(np.pi) ) #ステップ数. ある程度多くないと時間間隔が広すぎて発散してしまう
    amplitude = 50 # 設定値　信号の強度
    frequency = 1500 # 設定値　1500 [MHz]: 信号の周波数
    phase = 0 # 位相　
    pulselength = 0.01 * abs(theta)/(np.pi) # 0.01 [us]: amp=50, freq=1500の時にπ回転に必要な時間( QuTipで求めた実験値) 
    return ( pulselength, nsteps, [amplitude, frequency, phase] )

y = calc_qstate( qtype=["sc_transmon",1500], ini_state=[[1+0j,0j]], wave_params= rx( theta ) )
pulselength = rx( np.pi )[0]
plot_qstate( y, pulselength )